In [1]:
# =========================================================
# ✅ Optimized Research Paper Implementation of Kruskal MST
# =========================================================
# Implements:
# - Batch processing (√q strategy)
# - Parallel sorting using joblib
# - Edge filtering optimization (solves long-running issue)
# - Uses |w - x| dynamic weights (as defined in paper)
# =========================================================

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from math import sqrt
from time import time


# --------------------------------------------------------
# ✅ DSU (Disjoint Set Union) for Kruskal
# --------------------------------------------------------
class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
        self.sz = [1] * n

    def find(self, x):
        while x != self.parent[x]:
            x = self.parent[x]
        return x

    def union(self, a, b):
        a, b = self.find(a), self.find(b)
        if a == b:
            return False
        if self.sz[a] < self.sz[b]:
            a, b = b, a
        self.parent[b] = a
        self.sz[a] += self.sz[b]
        return True


# --------------------------------------------------------
# ✅ Edge Filtering + Parallel Sort (fixes performance)
# --------------------------------------------------------
def parallel_sort_edges(edges, x, threshold=100):
    """
    Optimization: instead of sorting entire 200,000 edges
    filter edges close to query x to reduce sorting time
    """

    # Select only edges where abs(weight - x) <= threshold
    filtered = [(u, v, w) for (u, v, w) in edges if abs(w - x) <= threshold]

    # Fallback to full edge list if filter becomes too small
    if len(filtered) < 500:
        filtered = edges

    return sorted(filtered, key=lambda e: abs(e[2] - x))


# --------------------------------------------------------
# ✅ Proposed Extended Kruskal MST
# --------------------------------------------------------
def proposed_mst(edges, n, queries):

    weights = [w for _, _, w in edges]
    unique_weight_ranges = sorted(set(weights))

    mst_results = []
    batch_size = max(2, int(sqrt(len(queries))))  # √q batching

    print("✅ Preprocessing complete.")
    print(f"✅ Unique weight ranges: {len(unique_weight_ranges)}")
    print(f"✅ Batch size for queries: {batch_size}\n")

    for batch_start in range(0, len(queries), batch_size):
        batch = queries[batch_start: batch_start + batch_size]

        sorted_batches = Parallel(n_jobs=-1)(
            delayed(parallel_sort_edges)(edges, x, threshold=100)
            for x in batch
        )

        for sorted_edges, x in zip(sorted_batches, batch):
            dsu = DSU(n)
            mst_cost = 0

            for u, v, w in sorted_edges:
                if dsu.union(u, v):
                    mst_cost += abs(w - x)

            mst_results.append(mst_cost)

    return mst_results


# --------------------------------------------------------
# ✅ Naive Kruskal — recompute MST for each query
# --------------------------------------------------------
def naive_kruskal_dynamic(edges, num_nodes, queries):
    results = []
    start = time()

    for x in queries:
        modified_edges = [(u, v, abs(w - x)) for (u, v, w) in edges]
        modified_edges.sort(key=lambda edge: edge[2])

        dsu = DSU(num_nodes)
        mst_cost = 0
        edges_used = 0

        for u, v, w in modified_edges:
            if dsu.union(u, v):
                mst_cost += w
                edges_used += 1
                if edges_used == num_nodes - 1:
                    break

        results.append(mst_cost)

    end = time()
    return results, end - start


# --------------------------------------------------------
# ✅ Load dataset (contains u, v, w columns)
# --------------------------------------------------------
df = pd.read_csv("generated_big_graph.csv").sample(500000)  # sample 200k edges for faster testing

edges = list(df.itertuples(index=False, name=None))
num_nodes = max(df["u"].max(), df["v"].max()) + 1

print("✅ Dataset loaded successfully!")
print(f"🔹 Total edges used: {len(edges)}")
print(f"🔹 Total nodes detected: {num_nodes}")
print(df.head())


# --------------------------------------------------------
# ✅ Generate 1000 random dynamic queries
# --------------------------------------------------------
queries = np.random.randint(1, 1000, size=1000).tolist()


# --------------------------------------------------------
# ✅ TIME COMPARISON
# --------------------------------------------------------
print("\n🔥 Running Naive Kruskal...")
naive_results, naive_time = naive_kruskal_dynamic(edges, num_nodes, queries)

print("\n⚡ Running Proposed Optimized Kruskal...")
start = time()
proposed_results = proposed_mst(edges, num_nodes, queries)
proposed_time = time() - start


# --------------------------------------------------------
# ✅ Results Output
# --------------------------------------------------------
print("\n====================================================")
print("                ⏱ TIME COMPARISON")
print("====================================================")
print(f"Naive Kruskal      : {naive_time:.4f} sec")
print(f"Proposed Algorithm : {proposed_time:.4f} sec")
print("====================================================\n")

print("🔹 Comparison of MST Costs (first 10 queries):")
for q, n_cost, p_cost in list(zip(queries, naive_results, proposed_results))[:10]:
    print(f"x={q} → Naive={n_cost}   Proposed={p_cost}")


✅ Dataset loaded successfully!
🔹 Total edges used: 500000
🔹 Total nodes detected: 5000
            u     v    w
1373936  4930   954  358
1388141  2061  1879  298
1861322  4961  2861  194
401030   1488   971  725
79778    3204   926  731

🔥 Running Naive Kruskal...

⚡ Running Proposed Optimized Kruskal...
✅ Preprocessing complete.
✅ Unique weight ranges: 999
✅ Batch size for queries: 31


                ⏱ TIME COMPARISON
Naive Kruskal      : 545.5677 sec
Proposed Algorithm : 409.4098 sec

🔹 Comparison of MST Costs (first 10 queries):
x=394 → Naive=14696   Proposed=14696
x=806 → Naive=14819   Proposed=14819
x=771 → Naive=15049   Proposed=15049
x=387 → Naive=14940   Proposed=14940
x=537 → Naive=14941   Proposed=14941
x=377 → Naive=14602   Proposed=14602
x=573 → Naive=15007   Proposed=15007
x=331 → Naive=15025   Proposed=15025
x=949 → Naive=15015   Proposed=15015
x=643 → Naive=15407   Proposed=15407
